<a href="https://colab.research.google.com/github/NikosAng/UEA-macro-lectures/blob/main/ueamacro1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install pandas_datareader ruptures pmdarima
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas_datareader import data as web
from statsmodels.tsa.stattools import adfuller, kpss, zivot_andrews, coint
from statsmodels.tsa.arima.model import ARIMA
plt.rcParams["figure.dpi"] = 110


### 2. Pull & Inspect Data

1. **Fetch real GDP**  
   We use the FRED series `GDPC1` (Real Gross Domestic Product) from 1950-01-01 onward.

2. **Align to quarter-ends**  
   To ensure a clean quarterly frequency, we resample with `.resample('Q').last()`, which takes the last observation within each calendar quarter.

3. **Quick sanity check**  
   - Display the first few rows to confirm the series is loaded correctly.  
   - Print the count of non-missing values to verify that we have a complete quarterly history.


In [ ]:
#@title 2. Pull & Inspect Data
# Fetch quarterly real GDP from FRED
gdp = web.DataReader("GDPC1", "fred", "1950-01-01")
# Resample to exact quarter-ends
gdp = gdp.resample("Q").last()

print("GDP head:")
display(gdp.head())
print("Non-missing count:", gdp.count().GDPC1)


### 3. Simulate TS vs DS (Intuition)

Before diving into real data, let’s build simple toy series to see the difference between:

- **Trend-Stationary (TS)**: fluctuations that revert to a fixed, deterministic trend.
- **Difference-Stationary (DS)**: a pure random walk with drift, where shocks permanently shift the level.

**What the code does:**
1. **Set a random seed** for reproducibility.
2. **TS series**  
   - Generate AR(1) noise with coefficient φ = 0.8.  
   - Add a deterministic linear trend (0.05 per period) to the noise.  
   - Results in a series that wanders but always pulls back toward the trend.
3. **DS series**  
   - Simulate a random walk with constant drift (0.02 per period).  
   - Each shock accumulates permanently.
4. **Plot both** on the same axes to visualise:  
   - The TS path hovers around its trend line.  
   - The DS path continuously drifts as shocks accumulate.

In [ ]:
#@title 3. Simulate TS vs DS (intuition)
np.random.seed(1)
T=200
# TS: linear trend + AR(1) noise
phi=0.8; u=np.zeros(T)
for t in range(1,T): u[t]=phi*u[t-1]+np.random.randn()*0.5
ts = 0.05*np.arange(T)+u
# DS: random walk with drift
rw = np.cumsum(0.02+np.random.randn(T)*0.5)

plt.plot(ts, label="TS (trend-stationary)")
plt.plot(rw, label="DS (unit-root)")
plt.legend(); plt.title("Simulated TS vs DS"); plt.show()


### 4. ADF & KPSS Helper + Unit-Root Tests

In this cell we:

1. **Define a helper function** `unitroot_tests(y, name)` that:
   - Drops any missing values.
   - Prints the sample size (`n`).
   - Runs the **Augmented Dickey–Fuller (ADF)** test for a unit root (`H₀: I(1)`).
   - Runs the **KPSS** test for stationarity (`H₀: I(0)`).
   - Reports both p-values side by side for quick comparison.

2. **Prepare our series**:
   - `log_gdp`: the natural log of real GDP (levels) with NAs removed.
   - `dlog_gdp`: the first difference of `log_gdp` (i.e. growth rates), also NA-cleaned.

3. **Run the tests** on each series:
   - If **ADF p-value < 0.05**, we reject the unit‐root null in favor of stationarity.
   - If **KPSS p-value < 0.05**, we reject the stationarity null in favor of a unit root.
   - Together, these give a more robust picture of whether the series is truly I(0) or I(1).


In [ ]:
#@title 4. ADF & KPSS Helper + Tests
def unitroot_tests(y, name):
    y = y.dropna()
    print(f"→ {name}:  n={len(y)} obs")
    adf_p = adfuller(y, autolag="AIC")[1]
    kpss_p= kpss(y, nlags="auto", regression="c")[1]
    print(f"    ADF  p-value = {adf_p:.3f}")
    print(f"    KPSS p-value = {kpss_p:.3f}\n")

# Prepare series
log_gdp  = np.log(gdp["GDPC1"]).dropna()
dlog_gdp = log_gdp.diff().dropna()

# Run tests
unitroot_tests(log_gdp,  "log GDP level")
unitroot_tests(dlog_gdp, "Δ log GDP growth")


### 5. Structural Break Test (Zivot–Andrews)

We now test whether the log-level GDP series has an endogenous break in its intercept (e.g. a sudden shift in mean) that could invalidate standard unit-root tests.

1. **Import** the `zivot_andrews` function.  
2. **Copy** the cleaned `log_gdp` series into `y`.  
3. **Run** `zivot_andrews(y, maxlag=8, regression='c')`, which:
   - Searches over all possible break dates (up to 8 lags from each end) to find the date that makes the null of a unit root hardest to reject.
   - Returns **five** values:
     - `za_stat`: the Zivot–Andrews test statistic.
     - `za_p`: its approximate p-value.
     - `za_crit`: a dictionary of critical values.
     - `za_lag`: the lag length actually used in the final regression.
     - `za_bp`: the integer index (position in the series) of the estimated break point.

4. **Print** all five to understand:
   - Whether there is strong evidence of a break (`p < 0.05`),  
   - The precise break quarter,  
   - And the critical thresholds against which the test is compared.


In [ ]:
# 5. Structural Break Test (Zivot–Andrews)
from statsmodels.tsa.stattools import zivot_andrews

y = log_gdp.copy()

# unpack all 5 returns: stat, p‐value, crit‐dict, used‐lag, break‐location
za_stat, za_p, za_crit, za_lag, za_bp = zivot_andrews(y, maxlag=8, regression="c")

print(f"Zivot–Andrews statistic = {za_stat:.3f}")
print(f"p-value                  = {za_p:.3f}")
print(f"critical values          = {za_crit}")
print(f"chosen lag               = {za_lag}")
print(f"break at index           = {za_bp}  → quarter = {y.index[za_bp].date()}")


### 6. AR(1) Fit → Half-Life & Impulse Response

In this cell we quantify how quickly GDP growth shocks decay using a simple AR(1) model:

1. **Fit an AR(1)**  
   - We model the growth series `dlog_gdp` as  
     \[
       y_t = \phi \, y_{t-1} + \varepsilon_t.
     \]
   - Use `statsmodels`’ `ARIMA(order=(1,0,0))` to estimate the coefficient \(\phi\).

2. **Compute the half-life**  
   - The half-life \(HL\) is the number of periods it takes for a one-unit shock to decay to 50% of its initial impact:
     \[
       HL = \frac{\ln(0.5)}{\ln|\phi|}.
     \]
   - Print the estimated \(\phi\) and the corresponding half-life in quarters.

3. **Plot the Impulse Response Function (IRF)**  
   - For an AR(1), the IRF at horizon \(h\) is simply \(\psi_h = \phi^h\).  
   - We stem-plot \(\psi_h\) for \(h = 0,1,\dots,19\) to visualize how the effect of a shock decays over time.


In [ ]:
#@title 6. AR(1) Fit → Half-Life & IRF (fixed stem call)
y = dlog_gdp.copy()
model = ARIMA(y, order=(1,0,0)).fit()
phi = model.params["ar.L1"]
hl  = np.log(0.5)/np.log(abs(phi))
print(f"Estimated φ = {phi:.3f}")
print(f"Half-Life ≈ {hl:.1f} quarters")

# IRF
H = 20
psi = phi**np.arange(H)
plt.stem(np.arange(H), psi, basefmt=" ")
plt.title("AR(1) Impulse Response")
plt.xlabel("horizon")
plt.ylabel("ψₕ")
plt.show()


Here we use the Variance‐Ratio (VR) test to gauge persistence without imposing an ARMA structure:

**1. Compute the VR statistic**  
For each horizon \(k = 1, \dots, 40\), we calculate  
\[
\mathrm{VR}(k) \;=\; \frac{\Var(y_t - y_{t-k})}{k \,\Var(\Delta y_t)}.
\]

- If \(y_t\) is a pure random walk (\(I(1)\)), then \(\mathrm{VR}(k)\approx1\) for all \(k\).  
- If \(y_t\) is stationary (\(I(0)\)), then \(\mathrm{VR}(k)\to0\) as \(k\to\infty\).  
- Intermediate values indicate a mixture of permanent and transitory components.

**2. Bootstrap a confidence band**  
- Resample the first differences \(\Delta y_t\) with replacement \(B=200\) times.  
- For each bootstrap sample, recompute the VR profile.  
- Take the 2.5 % and 97.5 % quantiles at each \(k\) to form a Monte Carlo confidence interval.

**3. Plot and interpret**  
- **Solid line**: the empirical VR curve.  
- **Shaded band**: the 95 % bootstrap confidence interval.  
- **Dashed horizontal line** at 1: the random-walk benchmark.  

If the VR curve lies significantly below 1 (outside the band) for large \(k\), that indicates strong mean reversion (stationarity). If it hovers near 1, the series behaves more like a unit root.


In [ ]:
#@title 7. Variance-Ratio Profile (with bootstrap CI)
def vr_profile(y, max_h=40, B=200):
    y = y.dropna().to_numpy()
    d = np.diff(y); v1 = np.var(d, ddof=0)
    vr = [np.var(y[h:]-y[:-h], ddof=0)/(h*v1) for h in range(1,max_h+1)]
    boot = []
    for _ in range(B):
        db = np.random.choice(d, size=len(d), replace=True)
        yb = np.insert(np.cumsum(db), 0, y[0])
        v1b = np.var(np.diff(yb), ddof=0)
        boot.append([np.var(yb[h:]-yb[:-h], ddof=0)/(h*v1b) for h in range(1,max_h+1)])
    ci = np.percentile(boot, [2.5,97.5], axis=0)
    return np.array(vr), ci

vr, ci = vr_profile(log_gdp)
h = np.arange(1,len(vr)+1)
plt.plot(h, vr, label="VR(k)")
plt.fill_between(h, ci[0], ci[1], color="gray", alpha=0.3)
plt.axhline(1, ls="--", color="k")
plt.title("Variance-Ratio Profile (log GDP)"); plt.xlabel("k"); plt.ylabel("VR"); plt.show()


### 9. Toy NK IRFs & Persistence Variation

To see how persistence alters policy‐relevant dynamics, we plot “toy” impulse‐response functions (IRFs) of a typical New-Keynesian variable (like the output gap or inflation) under three different persistence parameters \(\rho\):

- **\(\rho=0.3\)**: low persistence — shocks die out quickly.  
- **\(\rho=0.6\)**: moderate persistence — shocks linger for several periods.  
- **\(\rho=0.9\)**: high persistence — shocks have long-lasting effects.

The IRF at horizon \(h\) is simply \(\rho^h\). By overlaying these curves, you can immediately see that:

- With **low \(\rho\)**, the response nearly vanishes within a few quarters.  
- With **high \(\rho\)**, the response remains substantially above zero even at long horizons.

This illustrates why accurate persistence measurement matters: policy rules (e.g. Taylor rules) must be calibrated to match the true inertia of the economy.  


In [ ]:
#@title 9. Toy NK IRFs & Persistence Variation
import matplotlib.pyplot as plt
H=20
# Base IRFs
for rho in [0.3, 0.6, 0.9]:
    irf = rho**np.arange(H)
    plt.plot(irf, label=f"ρ={rho}")
plt.axhline(0, color='k', lw=0.5)
plt.title("Toy NK IRFs (output gap or inflation)"); plt.xlabel("quarters")
plt.legend(); plt.show()
